In [2]:
from pprint import pprint

import pandas as pd
from fpdf import FPDF

In [4]:
novel_df = pd.read_csv("../data/novel_content_450000.csv")
meta_df = pd.read_csv("../data/novel_metadata_450000.csv")

In [5]:
# 연재수가 0인 데이터는 drop
drop_id = meta_df[meta_df["연재수"] == 0]["id"]

In [6]:
novel_df[novel_df["id"].isin(drop_id)]

,id,chapter,subinfo,tcontent


In [7]:
# novel에는 해당 id의 데이터가 없으므로, meta_df에서만 제거
meta_df.drop(index=meta_df[meta_df["id"].isin(drop_id)].index, inplace=True)

### 조회수

In [8]:
meta_df["조회수"].describe()

count      2027.000000
mean       3673.324618
std       23488.134152
min           0.000000
25%          10.000000
50%          52.000000
75%         391.000000
max      571542.000000
Name: 조회수, dtype: float64

In [9]:
hits_1 = len(meta_df[meta_df["조회수"] >= 1000]) # 1000 이상
hits_2 = len(meta_df[(meta_df["조회수"] >= 1000) & (meta_df["조회수"] < 2000)]) # 1000 이상 2000 미만
print(f"조회수 1000 이상 {hits_1}")
print(f"조회수 1000 이상 2000 미만 : {hits_2}")

조회수 1000 이상 371
조회수 1000 이상 2000 미만 : 97


In [10]:
meta_df[meta_df["조회수"] >= 3673].sort_values("조회수")

,title,author,작품등록일,최근연재일,연재수,조회수,추천수,글자수,id
1756,인터넷 숨기고 문명창조,백댕,2025.01.07 21:50,2025.01.22 20:45,32,3682,211,165544,453307
1735,퇴직 후 드래곤을 주움,신화필,2025.01.07 19:29,2025.01.22 18:40,16,3752,80,113285,453270
3059,갑자기 S급 각성자가 되었다,콜라탄산,2025.01.20 17:56,2025.01.22 13:05,8,3777,91,39583,455364
1956,선협에서 뽑기로 살아남는 법,백춘식,2025.01.09 20:08,2025.01.22 18:20,13,3801,124,89714,453642
243,EX급 신규 특성으로 초월급 헌터,GMMSE,2024.12.20 16:41,2025.01.02 11:20,14,3879,100,75743,450470
...,...,...,...,...,...,...,...,...,...
990,이민 갔는데 딸이 생겼다,소설마표,2024.12.30 13:41,2025.01.21 15:00,23,171914,3350,121034,451890
4,버림받은 재벌집 데릴사위는 땅부자가 되었다,류다™,2024.12.17 14:10,2025.01.22 08:05,38,269872,4577,196804,450006
1228,망한 세계를 게임으로 착각시켰다.,크레도,2025.01.01 23:21,2025.01.22 17:20,24,296989,10862,157207,452318
469,히든 특성으로 창술천재,황금비둘기,2024.12.23 14:47,2025.01.21 19:20,30,560085,10892,159654,450879


In [28]:
novel = novel_df[novel_df["id"].isin(meta_df[meta_df["조회수"] >= 3673]["id"].to_list())]

n = 5
pprint(meta_df[meta_df["id"] == novel["id"].iloc[n]])

                    title author             작품등록일             최근연재일  연재수  \
14  현생 풀템한 천재 투자자는 인생이 쉬움     앤조  2024.12.17 16:02  2025.01.13 18:00   29   

     조회수  추천수     글자수      id  
14  6859  208  147960  450023  


In [19]:
# PDF 출력
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# 한글 사용 시 폰트 추가(예: NotoSansKR-Regular.ttf)
pdf.add_font("NotoSansKR-Regular", "", "./NotoSansKR-Regular.ttf", uni=True)
pdf.set_font("NotoSansKR-Regular", "", 12)

# 각 행마다 새로운 페이지를 추가하고 내용 출력
for idx, row in novel[:100].iterrows():
    pdf.add_page()
    
    # 간단한 텍스트는 cell() 메서드로
    pdf.cell(0, 10, f"id: {row['id']}", ln=True)
    pdf.cell(0, 10, f"chapter: {row['chapter']}", ln=True)
    pdf.cell(0, 10, f"subinfo: {row['subinfo']}", ln=True)
    
    # 여러 줄(긴 텍스트)일 경우 multi_cell()을 사용
    pdf.multi_cell(0, 10, f"tcontent: {row['tcontent']}")

# PDF 파일 저장
pdf.output("../조회수_100.pdf")

/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33033
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33089
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33299
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33451
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33551
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:

''

#### 추천수

In [73]:
meta_df["추천수"].describe()

count     2027.000000
mean        91.558954
std        536.440159
min          0.000000
25%          0.000000
50%          0.000000
75%         13.000000
max      11599.000000
Name: 추천수, dtype: float64

In [74]:
meta_df[meta_df["추천수"] >= 91].sort_values("추천수")

,title,author,작품등록일,최근연재일,연재수,조회수,추천수,글자수,id
3059,갑자기 S급 각성자가 되었다,콜라탄산,2025.01.20 17:56,2025.01.22 13:05,8,3777,91,39583,455364
2013,삼국지 : 패왕지로,인더정글,2025.01.10 15:39,2025.01.20 19:00,11,2637,94,62735,453763
940,S급들에게 사랑받는 서포터가 되었다,재즈소울,2024.12.29 19:34,2025.01.05 16:21,12,5541,95,71444,451759
91,마안의 무신(仙斬神殺鬼滅錄),급로불사,2024.12.18 14:35,2025.01.21 17:35,30,5042,96,185250,450188
2138,천재 대학교수가 여제자와 하룻밤을 보냈다.,달Lee,2025.01.11 23:48,2025.01.22 21:50,13,6576,96,68891,453969
...,...,...,...,...,...,...,...,...,...
935,게임 속 미궁도시에서 살아남기,김시영,2024.12.29 17:05,2025.01.21 19:50,26,114796,3829,140736,451741
4,버림받은 재벌집 데릴사위는 땅부자가 되었다,류다™,2024.12.17 14:10,2025.01.22 08:05,38,269872,4577,196804,450006
1228,망한 세계를 게임으로 착각시켰다.,크레도,2025.01.01 23:21,2025.01.22 17:20,24,296989,10862,157207,452318
469,히든 특성으로 창술천재,황금비둘기,2024.12.23 14:47,2025.01.21 19:20,30,560085,10892,159654,450879


#### 연재수 당 조회수

In [31]:
meta_df["연재당조회수"] = meta_df["조회수"] / meta_df["연재수"]

In [32]:
meta_df["연재당조회수"].describe()

count     2027.000000
mean       171.839280
std        820.557910
min          0.000000
25%          4.000000
50%          9.000000
75%         43.678571
max      18669.500000
Name: 연재당조회수, dtype: float64

In [33]:
meta_df[meta_df["연재당조회수"] > 171].sort_values("연재당조회수").head(15)

,title,author,작품등록일,최근연재일,연재수,조회수,추천수,글자수,id,연재당조회수
2397,"[개정판]천마, 재벌되다",카이젠,2025.01.14 11:43,2025.01.22 20:15,11,1894,55,38199,454381,172.181818
1612,폐관수련했더니 사람이 달라짐,무령.,2025.01.06 16:46,2025.01.17 08:00,14,2445,67,82959,453049,174.642857
1636,용의 힘으로 탑 등반,샤라웃,2025.01.06 20:02,2025.01.22 01:17,17,2972,84,98549,453085,174.823529
148,초월급 권능을 손에 넣었다,관심태그,2024.12.19 09:00,2024.12.21 12:20,5,875,34,20696,450287,175.000000
1540,내 던전이 생김,글로틴,2025.01.05 22:57,2025.01.22 19:00,19,3326,120,117553,452920,175.052632
947,무한 복제능력으로 자동사냥,피초,2024.12.29 21:23,2025.01.21 19:20,25,4403,209,144261,451778,176.120000
189,요리사가 무림에서 살아 남는법,장진백,2024.12.19 19:39,2024.12.28 19:00,11,1938,48,80657,450365,176.181818
634,내가 대통령이다.,나정치,2024.12.25 21:07,2025.01.22 08:00,27,4778,177,140558,451197,176.962963
719,인종차별로 흑화한 축구 감독이 4부리그에서 전설 씀,이고난,2024.12.26 16:28,2025.01.07 17:00,14,2516,53,85590,451332,179.714286
1673,사천당문 기생오라비가 되었다,서오개,2025.01.07 07:37,2025.01.22 08:00,16,2896,70,99622,453148,181.000000


In [34]:
meta_df[meta_df["연재당조회수"] > 171].sort_values("연재당조회수")[-1::-1].head(15)

,title,author,작품등록일,최근연재일,연재수,조회수,추천수,글자수,id,연재당조회수
469,히든 특성으로 창술천재,황금비둘기,2024.12.23 14:47,2025.01.21 19:20,30,560085,10892,159654,450879,18669.500000
1463,악마 벨라시오,공허한인생,2025.01.05 01:47,2025.01.20 10:09,8,110128,20,23545,452785,13766.000000
1228,망한 세계를 게임으로 착각시켰다.,크레도,2025.01.01 23:21,2025.01.22 17:20,24,296989,10862,157207,452318,12374.541667
49,회귀한 투수에게 흑역사란 없다,한지훈,2024.12.17 22:52,2025.01.22 07:20,57,571542,11599,318349,450084,10027.052632
990,이민 갔는데 딸이 생겼다,소설마표,2024.12.30 13:41,2025.01.21 15:00,23,171914,3350,121034,451890,7474.521739
4,버림받은 재벌집 데릴사위는 땅부자가 되었다,류다™,2024.12.17 14:10,2025.01.22 08:05,38,269872,4577,196804,450006,7101.894737
1011,EX급 약초밭을 상속받았다,계란탁,2024.12.30 17:14,2025.01.22 13:50,23,135670,3732,130248,451926,5898.695652
1354,방출된 연습생이 음악으로 찢어,강선우,2025.01.03 15:25,2025.01.22 18:00,20,117462,2840,130780,452608,5873.100000
1720,섬유의 신,영완(映完),2025.01.07 17:56,2025.01.22 19:20,17,79773,3108,86925,453239,4692.529412
315,재능있는 헌터는 마정석을 먹고 성장한다,bums™,2024.12.21 14:42,2025.01.21 21:20,34,159213,3471,198800,450605,4682.735294


#### 연재수 당 추천수

In [38]:
meta_df["연재당추천수"] = meta_df["추천수"] / meta_df["연재수"]

In [39]:
meta_df["연재당추천수"].describe()

count    2027.000000
mean        4.435491
std        19.508211
min         0.000000
25%         0.000000
50%         0.000000
75%         1.674242
max       452.583333
Name: 연재당추천수, dtype: float64

In [40]:
meta_df[meta_df["연재당추천수"] > 4].sort_values("연재당추천수").head(15)

,title,author,작품등록일,최근연재일,연재수,조회수,추천수,글자수,id,연재당조회수,연재당추천수
978,꿈을 만드는 야매 작가,클레어킨,2024.12.30 10:37,2025.01.21 18:15,21,1373,85,125489,451862,65.380952,4.047619
1712,점소이가 소환을 잘 함,방심,2025.01.07 16:41,2025.01.22 20:20,20,1916,81,109150,453221,95.800000,4.050000
1782,천재 배우는 사랑으로 자라난다,도해서,2025.01.08 03:24,2025.01.22 20:23,12,604,49,65143,453349,50.333333,4.083333
591,결국 살아남아야하니까,흑야아,2024.12.25 10:05,2025.01.22 08:10,19,1153,78,87146,451121,60.684211,4.105263
2190,좀비 아포칼립스에서 사이보그가 되다,방기만구,2025.01.12 16:26,2025.01.22 22:20,13,775,54,68322,454049,59.615385,4.153846
553,회귀했더니 391레벨,이파민,2024.12.24 19:09,2025.01.08 17:50,18,3589,75,110202,451056,199.388889,4.166667
2925,천공성 라퓨타의 주인이 되었다,장진백,2025.01.19 16:43,2025.01.22 20:20,5,399,21,29858,455181,79.800000,4.200000
2066,수선자가 선협 아카데미의 교수가 되었다,검문연,2025.01.11 02:30,2025.01.22 15:05,13,3988,55,75239,453857,306.769231,4.230769
1623,이혼 후 EX급 분대장으로 각성했다,준시최,2025.01.06 18:46,2025.01.12 13:34,8,663,34,34327,453069,82.875000,4.250000
429,유니폼 수집으로 SSS급 스트라이커,노대표,2024.12.23 07:53,2025.01.08 13:00,19,2261,81,137821,450813,119.000000,4.263158


In [82]:
# 연재당추천수와 일반 추천수 실제 비교
meta_df[meta_df["연재당추천수"] > 4].sort_values("연재당추천수").iloc[-1::-1].head(15)

,title,author,작품등록일,최근연재일,연재수,조회수,추천수,글자수,id,연재당조회수,연재당추천수
1228,망한 세계를 게임으로 착각시켰다.,크레도,2025.01.01 23:21,2025.01.22 17:20,24,296989,10862,157207,452318,12374.541667,452.583333
469,히든 특성으로 창술천재,황금비둘기,2024.12.23 14:47,2025.01.21 19:20,30,560085,10892,159654,450879,18669.500000,363.066667
49,회귀한 투수에게 흑역사란 없다,한지훈,2024.12.17 22:52,2025.01.22 07:20,57,571542,11599,318349,450084,10027.052632,203.491228
1720,섬유의 신,영완(映完),2025.01.07 17:56,2025.01.22 19:20,17,79773,3108,86925,453239,4692.529412,182.823529
1011,EX급 약초밭을 상속받았다,계란탁,2024.12.30 17:14,2025.01.22 13:50,23,135670,3732,130248,451926,5898.695652,162.260870
935,게임 속 미궁도시에서 살아남기,김시영,2024.12.29 17:05,2025.01.21 19:50,26,114796,3829,140736,451741,4415.230769,147.269231
990,이민 갔는데 딸이 생겼다,소설마표,2024.12.30 13:41,2025.01.21 15:00,23,171914,3350,121034,451890,7474.521739,145.652174
1590,이순신 부랄 친구,Cider.cap,2025.01.06 14:13,2025.01.22 18:00,20,72058,2899,99207,453012,3602.900000,144.950000
1354,방출된 연습생이 음악으로 찢어,강선우,2025.01.03 15:25,2025.01.22 18:00,20,117462,2840,130780,452608,5873.100000,142.000000
1624,지옥에서 돌아왔더니 악역이 너무 쉽다,shoro,2025.01.06 18:47,2025.01.22 08:20,18,81057,2534,105992,453070,4503.166667,140.777778


In [83]:
meta_df[meta_df["추천수"] > 4].sort_values("추천수").iloc[-1::-1].head(15)

,title,author,작품등록일,최근연재일,연재수,조회수,추천수,글자수,id,연재당조회수,연재당추천수
49,회귀한 투수에게 흑역사란 없다,한지훈,2024.12.17 22:52,2025.01.22 07:20,57,571542,11599,318349,450084,10027.052632,203.491228
469,히든 특성으로 창술천재,황금비둘기,2024.12.23 14:47,2025.01.21 19:20,30,560085,10892,159654,450879,18669.500000,363.066667
1228,망한 세계를 게임으로 착각시켰다.,크레도,2025.01.01 23:21,2025.01.22 17:20,24,296989,10862,157207,452318,12374.541667,452.583333
4,버림받은 재벌집 데릴사위는 땅부자가 되었다,류다™,2024.12.17 14:10,2025.01.22 08:05,38,269872,4577,196804,450006,7101.894737,120.447368
935,게임 속 미궁도시에서 살아남기,김시영,2024.12.29 17:05,2025.01.21 19:50,26,114796,3829,140736,451741,4415.230769,147.269231
1011,EX급 약초밭을 상속받았다,계란탁,2024.12.30 17:14,2025.01.22 13:50,23,135670,3732,130248,451926,5898.695652,162.260870
315,재능있는 헌터는 마정석을 먹고 성장한다,bums™,2024.12.21 14:42,2025.01.21 21:20,34,159213,3471,198800,450605,4682.735294,102.088235
990,이민 갔는데 딸이 생겼다,소설마표,2024.12.30 13:41,2025.01.21 15:00,23,171914,3350,121034,451890,7474.521739,145.652174
1720,섬유의 신,영완(映完),2025.01.07 17:56,2025.01.22 19:20,17,79773,3108,86925,453239,4692.529412,182.823529
323,선협 속 EX급 특성으로 수련천재,꿀잼글,2024.12.21 16:40,2025.01.21 18:20,33,142336,3053,97309,450620,4313.212121,92.515152


#### PDF로 출력

In [ ]:
# 조회수 PDF
novel = novel_df[novel_df["id"].isin(meta_df[meta_df["조회수"] >= 3673]["id"].to_list())]

n = 5
pprint(meta_df[meta_df["id"] == novel["id"].iloc[n]])

                    title author             작품등록일             최근연재일  연재수  \
14  현생 풀템한 천재 투자자는 인생이 쉬움     앤조  2024.12.17 16:02  2025.01.13 18:00   29   

     조회수  추천수     글자수      id  
14  6859  208  147960  450023  


In [ ]:
# PDF 출력
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# 한글 사용 시 폰트 추가(예: NotoSansKR-Regular.ttf)
pdf.add_font("NotoSansKR-Regular", "", "./NotoSansKR-Regular.ttf", uni=True)
pdf.set_font("NotoSansKR-Regular", "", 12)

# 각 행마다 새로운 페이지를 추가하고 내용 출력
for idx, row in novel[:100].iterrows():
    pdf.add_page()
    
    # 간단한 텍스트는 cell() 메서드로
    pdf.cell(0, 10, f"id: {row['id']}", ln=True)
    pdf.cell(0, 10, f"chapter: {row['chapter']}", ln=True)
    pdf.cell(0, 10, f"subinfo: {row['subinfo']}", ln=True)
    
    # 여러 줄(긴 텍스트)일 경우 multi_cell()을 사용
    pdf.multi_cell(0, 10, f"tcontent: {row['tcontent']}")

# PDF 파일 저장
pdf.output("../조회수_100.pdf")

/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33033
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33089
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33299
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33451
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33551
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:

''

In [ ]:
# 추천수 PDF
novel = novel_df[novel_df["id"].isin(meta_df[meta_df["추천수"] >= 91]["id"].to_list())]

n = 6
pprint(meta_df[meta_df["id"] == novel["id"].iloc[n]])

                   title author             작품등록일             최근연재일  연재수  \
17  좀비 아포칼립스에서 방공호를 찾아다님    글러팅  2024.12.17 17:03  2025.01.21 21:05   32   

     조회수  추천수     글자수      id  
17  1950  109  187790  450030  


In [ ]:
# PDF 출력
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# 한글 사용 시 폰트 추가(예: NotoSansKR-Regular.ttf)
pdf.add_font("NotoSansKR-Regular", "", "./NotoSansKR-Regular.ttf", uni=True)
pdf.set_font("NotoSansKR-Regular", "", 12)

# 각 행마다 새로운 페이지를 추가하고 내용 출력
for idx, row in novel[:100].iterrows():
    pdf.add_page()
    
    # 간단한 텍스트는 cell() 메서드로
    pdf.cell(0, 10, f"id: {row['id']}", ln=True)
    pdf.cell(0, 10, f"chapter: {row['chapter']}", ln=True)
    pdf.cell(0, 10, f"subinfo: {row['subinfo']}", ln=True)
    
    # 여러 줄(긴 텍스트)일 경우 multi_cell()을 사용
    pdf.multi_cell(0, 10, f"tcontent: {row['tcontent']}")

# PDF 파일 저장
pdf.output("../조회수_100.pdf")

/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33033
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33089
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33299
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33451
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33551
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:

''

In [ ]:
# 연재당조회수 PDF
novel = novel_df[novel_df["id"].isin(meta_df[meta_df["연재당조회수"] > 171]["id"].to_list())]

n = 10
pprint(meta_df[meta_df["id"] == novel["id"].iloc[n]])

              title author             작품등록일             최근연재일  연재수    조회수  \
72  회귀 후 300억으로 요식업     만선  2024.12.18 09:45  2025.01.21 23:32   38  39289   

    추천수     글자수      id       연재당조회수  
72  660  198815  450136  1033.921053  


In [ ]:
# PDF 출력
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# 한글 사용 시 폰트 추가(예: NotoSansKR-Regular.ttf)
pdf.add_font("NotoSansKR-Regular", "", "./NotoSansKR-Regular.ttf", uni=True)
pdf.set_font("NotoSansKR-Regular", "", 12)

# 각 행마다 새로운 페이지를 추가하고 내용 출력
for idx, row in novel[:100].iterrows():
    pdf.add_page()
    
    # 간단한 텍스트는 cell() 메서드로
    pdf.cell(0, 10, f"id: {row['id']}", ln=True)
    pdf.cell(0, 10, f"chapter: {row['chapter']}", ln=True)
    pdf.cell(0, 10, f"subinfo: {row['subinfo']}", ln=True)
    
    # 여러 줄(긴 텍스트)일 경우 multi_cell()을 사용
    pdf.multi_cell(0, 10, f"tcontent: {row['tcontent']}")

# PDF 파일 저장
pdf.output("../연재당조회수_100.pdf")

/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33033
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33089
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33299
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33451
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33551
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:

''

In [95]:
# 1화 기준 편당 추천수 상위 100개
novel_100 = novel_df[novel_df["id"].isin(meta_df[meta_df["연재당추천수"] > 4].sort_values("연재당추천수").iloc[-1:-111:-1]["id"])][novel_df["chapter"] == 1]

/tmp/ipykernel_2060017/1355888245.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  novel_100 = novel_df[novel_df["id"].isin(meta_df[meta_df["연재당추천수"] > 4].sort_values("연재당추천수").iloc[-1:-111:-1]["id"])][novel_df["chapter"] == 1]


In [96]:
# csv 출력
novel_100.to_csv("../likeperhits.csv")

In [99]:
# PDF 출력
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# 한글 사용 시 폰트 추가(예: NotoSansKR-Regular.ttf)
pdf.add_font("NotoSansKR-Regular", "", "./NotoSansKR-Regular.ttf", uni=True)
pdf.set_font("NotoSansKR-Regular", "", 12)

# 각 행마다 새로운 페이지를 추가하고 내용 출력
for idx, row in novel_100.iterrows():
    pdf.add_page()
    
    # 간단한 텍스트는 cell() 메서드로
    pdf.cell(0, 10, f"id: {row['id']}", ln=True)
    pdf.cell(0, 10, f"chapter: {row['chapter']}", ln=True)
    pdf.cell(0, 10, f"subinfo: {row['subinfo']}", ln=True)
    
    # 여러 줄(긴 텍스트)일 경우 multi_cell()을 사용
    pdf.multi_cell(0, 10, f"tcontent: {row['tcontent']}")

# PDF 파일 저장
pdf.output("../연재당추천수_상위100.pdf")

/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33103
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33301
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33566
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -34059
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -34066
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:

''

In [97]:
# 1화 기준 추천수 상위 100개
novel_100 = novel_df[novel_df["id"].isin(meta_df[meta_df["추천수"] > 91].sort_values("추천수").iloc[-1:-106:-1]["id"])][novel_df["chapter"] == 1]

/tmp/ipykernel_2060017/3880570324.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  novel_100 = novel_df[novel_df["id"].isin(meta_df[meta_df["추천수"] > 91].sort_values("추천수").iloc[-1:-106:-1]["id"])][novel_df["chapter"] == 1]


In [98]:
# csv 출력
novel_100.to_csv("../like.csv")

In [55]:
# PDF 출력
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# 한글 사용 시 폰트 추가(예: NotoSansKR-Regular.ttf)
pdf.add_font("NotoSansKR-Regular", "", "./NotoSansKR-Regular.ttf", uni=True)
pdf.set_font("NotoSansKR-Regular", "", 12)

# 각 행마다 새로운 페이지를 추가하고 내용 출력
for idx, row in novel_100.iterrows():
    pdf.add_page()
    
    # 간단한 텍스트는 cell() 메서드로
    pdf.cell(0, 10, f"id: {row['id']}", ln=True)
    pdf.cell(0, 10, f"chapter: {row['chapter']}", ln=True)
    pdf.cell(0, 10, f"subinfo: {row['subinfo']}", ln=True)
    
    # 여러 줄(긴 텍스트)일 경우 multi_cell()을 사용
    pdf.multi_cell(0, 10, f"tcontent: {row['tcontent']}")

# PDF 파일 저장
pdf.output("../추천수_상위100.pdf")

/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -32775
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -32777
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -33099
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -32808
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -32809
  warnings.warn("cmap value too big/small: %s" % cm)
/data/ephemeral/home/camper/.local/lib/python3.10/site-packages/fpdf/ttfonts.py:

''